In [12]:
import tensorflow as tf
import sklearn.metrics as sk

def eval_hyperparams(n_hidden=50, n_layers=1, learning_rate=.001,
                           dropout_prob=0.5, n_epochs=45, batch_size=1000,
                           weight_positives=True):
    
    print("---------------------------------------------")
    print("Model hyperparameters")
    print("n_hidden = %d" % n_hidden)
    print("n_layers = %d" % n_layers)
    print("learning_rate = %f" % learning_rate)
    print("n_epochs = %d" % n_epochs)
    print("batch_size = %d" % batch_size)
    print("weight_positives = %s" % str(weight_positives))
    print("dropout_prob = %f" % dropout_prob)
    print("---------------------------------------------")
    
    d = 784
    graph = tf.Graph()
    with graph.as_default():
        (train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

        # Reshaping the array to 4-dims so that it can work with the Keras API
        train_x = train_x.reshape(train_x.shape[0], 784)
        test_x = test_x.reshape(test_x.shape[0], 784)

        # Making sure that the values are float so that we can get decimal points after division
        train_x = train_x.astype('float32')
        test_x = test_x.astype('float32')
        
        # Normalizing the RGB codes by dividing it to the max RGB value.
        train_x /= 255
        test_x /= 255

        # Generate tensorflow graph
        with tf.name_scope("placeholders"):
            x = tf.placeholder(tf.float32, (None, d))
            y = tf.placeholder(tf.float32, (None,))
            w = tf.placeholder(tf.float32, (None,))
            keep_prob = tf.placeholder(tf.float32)
        for layer in range(n_layers):
            with tf.name_scope("layer-%d" % layer):
                W = tf.Variable(tf.random_normal((d, n_hidden)))
                b = tf.Variable(tf.random_normal((n_hidden,)))
                x_hidden = tf.nn.relu(tf.matmul(x, W) + b)
                # Apply dropout
                x_hidden = tf.nn.dropout(x_hidden, keep_prob)
        with tf.name_scope("output"):
            W = tf.Variable(tf.random_normal((n_hidden, 1)))
            b = tf.Variable(tf.random_normal((1,)))
            y_logit = tf.matmul(x_hidden, W) + b
            # the sigmoid gives the class probability of 1
            y_one_prob = tf.sigmoid(y_logit)
            # Rounding P(y=1) will give the correct prediction.
            y_pred = tf.round(y_one_prob)
        with tf.name_scope("loss"):
            # Compute the cross-entropy term for each datapoint
            y_expand = tf.expand_dims(y, 1)
            entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_logit, labels=y_expand)
#             # Multiply by weights
#             if weight_positives:
#                 w_expand = tf.expand_dims(w, 1)
#                 entropy = w_expand * entropy
            # Sum all contributions
            l = tf.reduce_sum(entropy**2)

        with tf.name_scope("optim"):
            train_op = tf.train.AdamOptimizer(learning_rate).minimize(l)

        hyperparam_str = "d-%d-hidden-%d-lr-%f-n_epochs-%d-batch_size-%d-weight_pos-%s" % (
            d, n_hidden, learning_rate, n_epochs, batch_size, str(weight_positives))
        N = train_x.shape[0]
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            step = 0
            for epoch in range(n_epochs):
                pos = 0
                while pos < N:
                    batch_X = train_x[pos:pos+batch_size]
                    batch_y = train_y[pos:pos+batch_size]
                    feed_dict = {x: batch_X, y: batch_y, keep_prob: dropout_prob}
                    _, loss = sess.run([train_op, l], feed_dict=feed_dict)
                    if (step % 5 == 0):
                        print("epoch %d, step %d, loss: %f" % (epoch, step, loss))
                    step += 1
                    pos += batch_size

            # Make Predictions (set keep_prob to 1.0 for predictions)
            test_y_pred = sess.run(y_pred, feed_dict={x: test_x, keep_prob: 1.0})

        weighted_score = sk.accuracy_score(test_y, test_y_pred)
        print("Valid Weighted Classification Accuracy: %f" % weighted_score)
    return weighted_score

if __name__ == "__main__":
      score = eval_hyperparams()


---------------------------------------------
Model hyperparameters
n_hidden = 50
n_layers = 1
learning_rate = 0.001000
n_epochs = 45
batch_size = 1000
weight_positives = True
dropout_prob = 0.500000
---------------------------------------------
epoch 0, step 0, loss: 87202400.000000
epoch 0, step 5, loss: 76459776.000000
epoch 0, step 10, loss: 54304912.000000
epoch 0, step 15, loss: 45205516.000000
epoch 0, step 20, loss: 49988572.000000
epoch 0, step 25, loss: 42179988.000000
epoch 0, step 30, loss: 45471116.000000
epoch 0, step 35, loss: 37261576.000000
epoch 0, step 40, loss: 33394884.000000
epoch 0, step 45, loss: 31296276.000000
epoch 0, step 50, loss: 30412180.000000
epoch 0, step 55, loss: 29236714.000000
epoch 1, step 60, loss: 27342348.000000
epoch 1, step 65, loss: 27847810.000000
epoch 1, step 70, loss: 24453852.000000
epoch 1, step 75, loss: 20952502.000000
epoch 1, step 80, loss: 27360068.000000
epoch 1, step 85, loss: 19555060.000000
epoch 1, step 90, loss: 23736836.000

epoch 16, step 990, loss: 522920.125000
epoch 16, step 995, loss: 524085.000000
epoch 16, step 1000, loss: 477812.062500
epoch 16, step 1005, loss: 577083.937500
epoch 16, step 1010, loss: 376508.250000
epoch 16, step 1015, loss: 441041.406250
epoch 17, step 1020, loss: 597320.875000
epoch 17, step 1025, loss: 354206.437500
epoch 17, step 1030, loss: 399911.500000
epoch 17, step 1035, loss: 484546.812500
epoch 17, step 1040, loss: 508130.468750
epoch 17, step 1045, loss: 443270.968750
epoch 17, step 1050, loss: 539542.187500
epoch 17, step 1055, loss: 351285.250000
epoch 17, step 1060, loss: 428697.312500
epoch 17, step 1065, loss: 372741.718750
epoch 17, step 1070, loss: 415228.312500
epoch 17, step 1075, loss: 329338.875000
epoch 18, step 1080, loss: 481071.312500
epoch 18, step 1085, loss: 371011.875000
epoch 18, step 1090, loss: 320539.187500
epoch 18, step 1095, loss: 384345.156250
epoch 18, step 1100, loss: 374835.125000
epoch 18, step 1105, loss: 448564.656250
epoch 18, step 111

epoch 33, step 1995, loss: 139885.875000
epoch 33, step 2000, loss: 119088.546875
epoch 33, step 2005, loss: 102525.867188
epoch 33, step 2010, loss: 188728.296875
epoch 33, step 2015, loss: 99522.484375
epoch 33, step 2020, loss: 121766.140625
epoch 33, step 2025, loss: 133892.531250
epoch 33, step 2030, loss: 105343.734375
epoch 33, step 2035, loss: 82786.250000
epoch 34, step 2040, loss: 128918.390625
epoch 34, step 2045, loss: 90317.281250
epoch 34, step 2050, loss: 98674.640625
epoch 34, step 2055, loss: 107518.117188
epoch 34, step 2060, loss: 118727.187500
epoch 34, step 2065, loss: 90695.500000
epoch 34, step 2070, loss: 203405.406250
epoch 34, step 2075, loss: 97660.296875
epoch 34, step 2080, loss: 85687.343750
epoch 34, step 2085, loss: 106600.476562
epoch 34, step 2090, loss: 76029.000000
epoch 34, step 2095, loss: 82386.953125
epoch 35, step 2100, loss: 106133.546875
epoch 35, step 2105, loss: 83232.156250
epoch 35, step 2110, loss: 70689.953125
epoch 35, step 2115, loss: 